In [1]:
!nvidia-smi

Sat Jan 24 23:38:47 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.57.6
Uninstalling transformers-4.57.6:
  Successfully uninstalled transformers-4.57.6
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
  Using cached transformers-4.57.6-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.57.6-py3-none-any.whl (12.0 MB)
Using cached accelerate-1.12.0-py3-none-any.whl (380 kB)


In [4]:
import torch
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device : ", device)

Device :  cuda


In [6]:
# from transformers import PegasusForConditionalGeneration
# # Check for GPU
# device = "cuda" if torch.cuda.is_available() else "cpu"

# model_name = "google/pegasus-xsum"
# model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# ARTICLE_TO_SUMMARIZE = [
#     "The quick brown fox jumps over the lazy dog.",
#     "Machine learning is a subset of artificial intelligence that focuses on algorithms and statistical models.",
#     "Natural language processing enables computers to understand and interpret human language.",
# ]

# # Note: We added padding=True for batch processing
# inputs = tokenizer(
#     ARTICLE_TO_SUMMARIZE,
#     max_length=1024,
#     return_tensors="pt",
#     truncation=True,
#     padding=True,
# ).to(device)

# # Generate Summary
# summary_ids = model.generate(inputs["input_ids"])
# summaries = tokenizer.batch_decode(
#     summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
# )

# print(summaries[0])

In [7]:
model = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)

print(f"Model loaded successfully on {device}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully on cuda


In [8]:
# Download the actual raw binary file
!wget https://github.com/SiddhuShkya/Text-Summarizer-With-HF/raw/main/data/summarizer-data.zip

# Now unzip will work
!unzip summarizer-data.zip

--2026-01-24 23:41:22--  https://github.com/SiddhuShkya/Text-Summarizer-With-HF/raw/main/data/summarizer-data.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/SiddhuShkya/Text-Summarizer-With-HF/main/data/summarizer-data.zip [following]
--2026-01-24 23:41:22--  https://raw.githubusercontent.com/SiddhuShkya/Text-Summarizer-With-HF/main/data/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4004830 (3.8M) [application/zip]
Saving to: ‘summarizer-data.zip.1’

summarizer-data.zip 100%[===================>]   3.82M  --.-KB/s    in 0.02s   

2026-01-24 23:41:23 (224 MB/s) - ‘s

In [9]:
train_df = pd.read_csv('./summarizer-data/train.csv')
test_df = pd.read_csv('./summarizer-data/test.csv')
val_df = pd.read_csv('./summarizer-data/validation.csv')

print("Features in the dataset: ", train_df.columns.tolist())
print("=" * 70)
print("Number of samples in each dataset:")
print("Train data samples: ", len(train_df))A
print("Test data samples: ", len(test_df))
print("Validation data samples: ", len(val_df))

Features in the dataset:  ['id', 'dialogue', 'summary']
Number of samples in each dataset:
Train data samples:  14731
Test data samples:  819
Validation data samples:  818


In [10]:
print(train_df["dialogue"][0])
print("\nSummary: ", train_df["summary"][0])

Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)

Summary:  Amanda baked cookies and will bring Jerry some tomorrow.


In [11]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(
        example_batch["dialogue"],
        max_length=512,
        truncation=True,
    )
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(
            example_batch["summary"],
            max_length=128,
            truncation=True,
        )
    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [12]:
from datasets import Dataset

# Convert dataframes → Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)
# Apply the function with .map()
train_dataset = train_dataset.map(convert_examples_to_features, batched=True)
test_dataset = test_dataset.map(convert_examples_to_features, batched=True)
val_dataset = val_dataset.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/14731 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4174: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [13]:
train_dataset

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14731
})

In [14]:
test_dataset

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 819
})

In [15]:
val_dataset

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 818
})

In [16]:
print("Train Dataset:\n", train_dataset)
print("Test Dataset:\n", test_dataset)
print("Val Dataset:\n", val_dataset)

Train Dataset:
 Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14731
})
Test Dataset:
 Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 819
})
Val Dataset:
 Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 818
})


In [17]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="pegasus-finetuned",
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=100000,
    gradient_accumulation_steps=16,
)

In [19]:
trainer = Trainer(
    model=model_pegasus,
    args=training_args,
    processing_class=tokenizer,
    data_collator=seq2seq_data_collator,
    train_dataset=test_dataset,
    eval_dataset=val_dataset,
)

In [20]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"
wandb: Using W&B in offline mode.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=52, training_loss=2.9682337779265184, metrics={'train_runtime': 549.9829, 'train_samples_per_second': 1.489, 'train_steps_per_second': 0.095, 'total_flos': 313176745058304.0, 'train_loss': 2.9682337779265184, 'epoch': 1.0})

In [21]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches"""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]


def calculate_metric_on_test_ds(
    dataset,
    metric,
    model,
    tokenizer,
    batch_size=1,
    device=device,
    column_text="article",
    column_summary="highlights",
):
    model.eval()  # 🔴 add

    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    with torch.no_grad():  # 🔴 add
        for article_batch, target_batch in zip(article_batches, target_batches):

            inputs = tokenizer(
                article_batch,
                max_length=256,          # 🔴 reduce
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )

            summaries = model.generate(
                input_ids=inputs["input_ids"].to(device),
                attention_mask=inputs["attention_mask"].to(device),
                max_new_tokens=128,
                num_beams=1,             # 🔴 critical
                do_sample=False,
                use_cache=True,
            )

            decoded_summaries = tokenizer.batch_decode(
                summaries, skip_special_tokens=True
            )

            metric.add_batch(
                predictions=decoded_summaries,
                references=target_batch,
            )

    return metric.compute()


In [22]:
!pip install evaluate

In [23]:
import evaluate

rouge_metric = evaluate.load("rouge")
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

In [24]:
score = calculate_metric_on_test_ds(
    dataset=val_dataset[0:10],
    metric=rouge_metric,
    model=trainer.model,
    tokenizer=tokenizer,
    column_text="dialogue",
    column_summary="summary"
)

rouge_dict = {name: score[name] for name in rouge_names}
pd.DataFrame(rouge_dict, index=[f'pegasus-finetuned'])

The following generation flags are not valid and may be ignored: ['length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


,rouge1,rouge2,rougeL,rougeLsum
pegasus-finetuned,0.305614,0.1115,0.247998,0.248458


In [25]:
model_pegasus.save_pretrained("pegasus-model")
tokenizer.save_pretrained("pegasus-tokenizer")

('pegasus-tokenizer/tokenizer_config.json',
 'pegasus-tokenizer/special_tokens_map.json',
 'pegasus-tokenizer/spiece.model',
 'pegasus-tokenizer/added_tokens.json',
 'pegasus-tokenizer/tokenizer.json')

In [50]:
model_path = "./pegasus-model"
tokenizer_path = "./pegasus-tokenizer"

In [55]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Load with local_files_only=True to prevent it from checking the internet
tokenizer = PegasusTokenizer.from_pretrained(tokenizer_path)
model = PegasusForConditionalGeneration.from_pretrained(model_path)

In [52]:
gen_kwargs = {
    "length_penalty": 0.8,
    "num_beams": 8,
    "max_length": 128,
}

In [53]:
sample_text = train_dataset[0]['dialogue']
reference = train_dataset[0]['summary']
pipe = pipeline("summarization", model="pegasus-model", tokenizer=tokenizer)

Device set to use cuda:0


In [54]:
print("Dialogue : \n", sample_text)
print("\nReference Summary : \n", reference)
print("\nModel Summary : \n", pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 25. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)


Dialogue : 
 Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)

Reference Summary : 
 Amanda baked cookies and will bring Jerry some tomorrow.

Model Summary : 
 Amanda: I'll bring you tomorrow :-) <n> Amanda: I baked cookies. Do you want some? <n> Amanda: I'll bring you tomorrow :-)


In [59]:
# 1. Grab your text from the dataset
sample_text = train_dataset[0]['dialogue']
reference = train_dataset[0]['summary']

# 2. Tokenize the input dialogue
# We use truncation=True to ensure it fits within the model's 1024 token limit
inputs = tokenizer(sample_text, truncation=True, padding="longest", return_tensors="pt")

# 3. Generate the summary
# The model produces token IDs
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=128,
    num_beams=4,
    length_penalty=2.0,
    early_stopping=True
)

# 4. Decode the IDs back into a string
decoded_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 5. Compare the results
print("--- DIALOGUE ---")
print(sample_text)
print("\n--- REFERENCE SUMMARY (Ground Truth) ---")
print(reference)
print("\n--- MODEL GENERATED SUMMARY ---")
print(decoded_summary)

--- DIALOGUE ---
Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)

--- REFERENCE SUMMARY (Ground Truth) ---
Amanda baked cookies and will bring Jerry some tomorrow.

--- MODEL GENERATED SUMMARY ---
Amanda: I baked cookies. Do you want some? Jerry: Sure! <n> Amanda: . <n> I'll bring you tomorrow :-), Jerry
